# Inference

In [1]:
import os
import pandas as pd
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLO model
model = YOLO("/data/ephemeral/home/ay_yolo/yolo11x/yolov11x_ay133/weights/best.pt")

# Define path to the image file
source = "/data/ephemeral/home/dataset/test"
output_csv = './submission5.csv'

# Initialize variables for submission
prediction_strings = []
file_names = []
score_threshold = 0.05  # Set your desired score threshold

In [2]:
# Iterate through each test image
for img_file in os.listdir(source):
    img_path = os.path.join(source, img_file)
    
    # Open image
    img = Image.open(img_path)

    # Run inference on the resized image
    results = model(img)  # Run inference
    
    prediction_string = ''
    
    # Iterate through detected boxes
    for result in results:  # results is a list of Result objects
        for box, score, label in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
            if score > score_threshold:
                # Convert YOLO box format [x1, y1, x2, y2] to PASCAL format
                x1 = box[0].item()  # x1
                y1 = box[1].item()  # y1
                x2 = box[2].item()  # x2
                y2 = box[3].item()  # y2
                
                # Append to prediction string in desired format
                # prediction_string += f"{int(label.item())} {score:.6f} {x_min:.4f} {y_min:.4f} {x_max:.4f} {y_max:.4f} "
                prediction_string += f"{int(label.item())} {score:.6f} {x1:.4f} {y1:.4f} {x2:.4f} {y2:.4f} "
    
    prediction_strings.append(prediction_string.strip())
    file_names.append(img_file)  # Use the file name directly


0: 640x640 1 Paper, 2 Plastics, 1 Plastic bag, 28.7ms
Speed: 7.8ms preprocess, 28.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Plastics, 1 Styrofoam, 29.5ms
Speed: 3.2ms preprocess, 29.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Plastic, 1 Plastic bag, 29.5ms
Speed: 3.0ms preprocess, 29.5ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 General trashs, 2 Papers, 1 Metal, 6 Plastics, 2 Plastic bags, 30.8ms
Speed: 3.0ms preprocess, 30.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 General trash, 1 Plastic bag, 28.9ms
Speed: 3.0ms preprocess, 28.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 General trash, 3 Papers, 28.8ms
Speed: 2.9ms preprocess, 28.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Paper, 3 Plastic bags, 29.0ms
Speed: 2.9ms preprocess, 29.0ms inference, 1.1ms 

In [5]:
# Create submission DataFrame
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission['image_id'] = 'test/'+ submission['image_id']
submission = submission.sort_values('image_id')

# Save to CSV
submission.to_csv(output_csv, index=False)
print(submission.head())

                                       PredictionString       image_id
1979  7 0.978529 602.3771 518.0337 956.4836 1023.661...  test/0000.jpg
3036     3 0.976416 343.5395 252.0934 753.4438 695.5673  test/0001.jpg
3193  1 0.960612 302.0970 306.5582 1023.7568 745.986...  test/0002.jpg
2943  9 0.940776 147.9106 260.3918 908.9467 819.7257...  test/0003.jpg
3079  1 0.895658 187.0345 253.0012 873.7788 778.1093...  test/0004.jpg
